### Common operations

- How to apply basic filters and transformations using AntsPy and SITK:
    - Denoise
    - Morphological operations
    - Shrink
    - Cropping
    - Padding
    - Blurring
    - Thresholding
    - Statistics

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


In [3]:
raw_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'sub-01', 'anat', 'sub-01_T1w.nii')
print(f'raw_img_path = {raw_img_path}')

raw_img_path = c:\Users\Windows\Desktop\GBM_Segmentation\Brain_Extraction\sub-01\anat\sub-01_T1w.nii


### AntsPy

#### Raw Image

In [4]:
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL') 

print(f'shape = {raw_img_ants.numpy().shape} -> (Z, X, Y)')

explore_3D_array(
    arr=raw_img_ants.numpy(),
    cmap='viridis'
)

shape = (320, 300, 208) -> (Z, X, Y)


interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Denoise

In [5]:
transformed = ants.denoise_image(raw_img_ants, shrink_factor=8)

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(),
    arr_after=transformed.numpy(),
    cmap='viridis'
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Morphological operations

In [6]:
"""
operation to apply
    "close" Morphological closing
    "dilate" Morphological dilation
    "erode" Morphological erosion
    "open" Morphological opening
"""

transformed = ants.morphology(raw_img_ants, radius=1, operation='erode', mtype='grayscale')

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(),
    arr_after=transformed.numpy(),
    cmap='viridis'
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

### Simple ITK

#### Raw Image

In [14]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk,'RPS')

print(f'shape = {sitk.GetArrayFromImage(raw_img_sitk).shape} -> (Z, X, Y)')
explore_3D_array(
    arr=sitk.GetArrayFromImage(raw_img_sitk),
    cmap='viridis'
)

shape = (320, 300, 208) -> (Z, X, Y)


interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Shrink

In [8]:
shrinkFactor = 3
transformed = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * raw_img_sitk.GetDimension() )

print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed))

shape before = (320, 300, 208)
shape after = (106, 100, 69)


interactive(children=(IntSlider(value=52, description='SLICE', max=105), Output()), _dom_classes=('widget-inte…

#### Crop

In [12]:
# Cropping takes the orientation of the pixels for the reference of lower & upper boundaries vectors
# Pixel orientation = RPS = (left-to-Right, anterior-to-Posterior, inferior-to-Superior)

# crop nothing
#transformed = sitk.Crop(raw_img_sitk)
#transformed = sitk.Crop(raw_img_sitk, (0,0,0), (0,0,0))

# crop 20 from left to right             X,Y,Z
#transformed = sitk.Crop(raw_img_sitk, (20,0,0), (0,0,0))

# crop 20 from left to right, crop 30 from anterior to posterior
#transformed = sitk.Crop(raw_img_sitk, (20,30,0), (0,0,0))

# crop 20 from left to right, crop 30 from anterior to posterior, 
# crop 10 from right to left, crop 5 from posterior to anterior. 
#transformed = sitk.Crop(raw_img_sitk, (20,30,0), (10,5,0)) 

# crop 40 from inferior to superior, crop 50 from superior to inferior
transformed = sitk.Crop(raw_img_sitk, (0,0,40), (0,0,50)) 


print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed))

shape before = (320, 300, 208)
shape after = (230, 300, 208)


interactive(children=(IntSlider(value=114, description='SLICE', max=229), Output()), _dom_classes=('widget-int…

#### Padding

In [15]:
constant = int(sitk.GetArrayFromImage(raw_img_sitk).min())
constant

0

In [16]:
# Padding (as Cropping) takes the orientation of the pixels for the reference of lower & upper boundaries vectors
# Pixel orientation = RPS = (left-to-Right, anterior-to-Posterior, inferior-to-Superior)

# pad nothing
#transformed = sitk.ConstantPad(raw_img_sitk)
#transformed = sitk.ConstantPad(raw_img_sitk,(0,0,0),(0,0,0), constant)

# pad 10 from left to right
#transformed = sitk.ConstantPad(raw_img_sitk,(10,0,0),(0,0,0),constant)

# pad 10 from left to right, pad 15 from anterior to posterior
#transformed = sitk.ConstantPad(raw_img_sitk,(10,15,0),(0,0,0),constant)

# pad 10 from left to right, pad 15 from anterior to posterior, 
# pad 5 from right to left, pad 8 from posterior to anterior. 
transformed = sitk.ConstantPad(raw_img_sitk,(10,15,0),(5,8,0),constant)


print(f'shape before = {sitk.GetArrayFromImage(raw_img_sitk).shape}')
print(f'shape after = {sitk.GetArrayFromImage(transformed).shape}')

explore_3D_array(sitk.GetArrayFromImage(transformed), cmap='viridis')

shape before = (320, 300, 208)
shape after = (320, 323, 223)


interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Denoise

Curvature Flow filter

In [17]:
transformed = sitk.CurvatureFlow(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='viridis'
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Morphological Operations

In [18]:
"""
sitk.GrayscaleMorphologicalClosing
sitk.GrayscaleDilate
sitk.GrayscaleErode
sitk.GrayscaleMorphologicalOpening

sitk.BinaryMorphologicalClosing
sitk.BinaryDilate
sitk.BinaryErode
sitk.BinaryMorphologicalOpening
"""

transformed = sitk.GrayscaleErode(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='viridis'
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Blurring

In [19]:
transformed = sitk.DiscreteGaussian(raw_img_sitk)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed),
    cmap='viridis'
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Thresholding

In [21]:
"""
sitk.OtsuThreshold
sitk.LiThreshold
sitk.TriangleThreshold
sitk.MomentsThreshold
"""

transformed = sitk.OtsuThreshold(raw_img_sitk, 0, 1)

explore_3D_array_comparison(
    arr_before=sitk.GetArrayFromImage(raw_img_sitk),
    arr_after=sitk.GetArrayFromImage(transformed)
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Statistics

In [22]:
stats = sitk.StatisticsImageFilter()
stats.Execute(raw_img_sitk)


print('\tRaw img')
print("min  =", stats.GetMinimum())
print("max  =", stats.GetMaximum())
print("mean =", stats.GetMean())


	Raw img
min  = 0.0
max  = 550.0
mean = 52.49758748998397
